# Iterativne metode
---

Za velike sustave, a posebno za sustave s malom ispunom (malo elemenata različitih od nule), te ukoliko je matrica sustava __strogo dijagonalno dominantna__, rješenje se može brzo naći __iterativnim metodama__
(vidi [Numerička matematika, poglavlje 3.8][RS04]):

__Definicija.__ Funkcija $F:\mathbb{R}^n\to \mathbb{R}^n$ je __kontrakcija__ ako postoji broj $q<1$ za koji vrijedi

$$
\| F(x)-F(y)\| < q\|x-y\|\qquad \forall x,y.
$$

__ Banachov teorem o fiksnoj točki.__ Ako je $F$ kontrakcija, tada niz definiran s

$$ 
x_{k+1}=F(x_k)
$$

konvergira prema jedinstvenom vektoru $\tilde x$ za kojeg vrijedi

$$
\tilde x = F(\tilde x).
$$

$\tilde x$ se zove __fiksna točka__ funkcije $F$. Za pogrešku u $k$-tom koraku vrijede ocjene

$$
\|x_k- \tilde x\| \leq \frac{q}{1-q} \|x_k-x_{k-1}\|
$$

i 

$$
\|x_k- \tilde x\| \leq \frac{q^k}{1-q} \|x_1-x_{0}\|,
$$

pri čemu je druga ocjena bolja. Brzina konvergencije je __linearna__,

$$
\|x_{k+1}-\tilde x\| \leq q\| x_k-\tilde x\|.
$$

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."


## Jacobi-jeva i Gauss-Seidel-ova metoda

Neka je 

$$F(x)=Bx+c,$$

pri čemu je $B$ nesignularna matrica. Tada je

$$
\| F(x)-F(y)\|=\| Bx+c-(By+c)\|=\|B(x-y)\| \leq \|B\| \|x-y\|,
$$

pa je $F$ kontrakcija ako ako je

$$
 \|B\|=q<1.
$$

Neka je zadan sustav  $Ax=b$. Matricu $A$ rastavimo kao

$$
A=D\,(L+I+U)
$$

pri čemu je $D$ dijagonalna matrica, $L$ strogo donje trokutasta matrica i $U$ strogo gornje trokutasta matrica.

### Jacobi-jeva metoda 

Neka je 

$$
B=-(L+U), \quad c=D^{-1}b.
$$


Ako je matrica $A$ _strogo dijagonalno dominantna_, 

$$
\| B\|_{\infty} = \max_i \sum_{{j=1} \atop {j\neq i}}^n \frac{|a_{ij}|}{|a_{ii}|}<1,
$$

tada je preslikavanje $F$ kontrakcija (moguće je uzeti i druge norme).

Niz 

$$
x_{k+1}=-(L+U)x_k+c
$$

očito konvergira prema rješenju sustava $x$.

### Gauss-Seidel-ova metoda 

Neka je 

$$
B=-(I+L)^{-1}U, \quad c=(I+L)\, D^{-1}b.
$$

Bez dokaza navodimo sljedeću tvrdnju: ako je matrica $A$ strogo dijagonalno dominantna,
onda je preslikavanje $F$ kontrakcija pa niz

$$
x_{k+1}=-(I+L)^{-1}Ux_k+(I+L)^{-1}D^{-1}b,
$$

odnosno

$$
x_{k+1}=-Lx_{k+1}-Ux_k+D^{-1}b,
$$

konvergira prema rješenju sustava $x$.

In [1]:
function myjacobi{T}(A::Array{T},b::Array{T},x::Array{T})
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    B=-(L+U)
    c=b./D
    q=norm(B,Inf)
    # @show q
    while d>tol
        y=B*x+c
        d=norm(x-y,Inf)
        # @show d
        x=y
    end
    x,d
end

myjacobi (generic function with 1 method)

In [2]:
n=8
A=rand(n,n)
# Napravimo matricu dijagonalno diminantnom
A=A+n*I
b=rand(n)

8-element Array{Float64,1}:
 0.787585 
 0.304544 
 0.503383 
 0.083224 
 0.101764 
 0.0891466
 0.908993 
 0.768695 

In [3]:
# Pocetni vektor
x0=rand(n)

8-element Array{Float64,1}:
 0.0376094
 0.774499 
 0.949289 
 0.764135 
 0.636331 
 0.486523 
 0.573785 
 0.203715 

In [4]:
x,d=myjacobi(A,b,x0)

([0.0830476, 0.0217063, 0.0434041, -0.014105, -0.0100403, -0.00676604, 0.0997835, 0.0870932], 1.0325074129013956e-13)

In [5]:
r=A*x-b

8-element Array{Float64,1}:
 3.48832e-13
 1.93734e-13
 2.95652e-13
 3.57228e-13
 2.62804e-13
 2.94778e-13
 2.85327e-13
 2.37921e-13

In [6]:
# Provjerimo i normu relativnog reziduala
norm(r)/(vecnorm(A)*norm(x))

2.0737077597515226e-13

In [7]:
function mygaussseidel{T}(A::Array{T},b::Array{T},x::Array{T})
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    # B=-inv(I+L)*U
    B=-(I+L)\U
    c=(I+L)\(b./D)
    # @show norm(U,Inf)
    y=Array{Float64}(n)
    while d>tol
        y=B*x+c
        d=norm(x-y)
        x=y
    end
    x,d
end

mygaussseidel (generic function with 1 method)

In [8]:
x,d=mygaussseidel(A,b,x0)

([0.0830476, 0.0217063, 0.0434041, -0.014105, -0.0100403, -0.00676604, 0.0997835, 0.0870932], 2.663952086507779e-14)

In [9]:
A*x-b

8-element Array{Float64,1}:
 -3.09752e-14
 -1.0103e-14 
  5.55112e-16
  2.02616e-15
  1.97065e-15
  1.04083e-15
  1.44329e-15
  1.11022e-16

Izmjerimo brzinu za veće matrice:

In [10]:
n=1024
A=rand(n,n)
# Napravimo matricu dijagonalno dominantnom
A=A+n*I
b=rand(n)
# Pocetni vektor
x0=rand(n)

1024-element Array{Float64,1}:
 0.754599 
 0.749356 
 0.286365 
 0.0525211
 0.318049 
 0.501857 
 0.731832 
 0.236568 
 0.217226 
 0.456119 
 0.430428 
 0.852485 
 0.647507 
 ⋮        
 0.525318 
 0.364415 
 0.482417 
 0.659323 
 0.224078 
 0.312743 
 0.368202 
 0.798221 
 0.161914 
 0.294896 
 0.65014  
 0.0260298

In [12]:
@time mygaussseidel(A,b,x0);

  0.163731 seconds (77 allocations: 64.413 MiB, 53.52% gc time)
